In [2]:
sys.path.insert(0, '/home/shengq2/program/spcount/src')

import pandas as pd
#from spcount.common_util import readFileMap

In [3]:
groupFile="/scratch/cqs/shengq2/spcount/fileList2.txt"
treeFile="/scratch/cqs/shengq2/spcount/RA_97_93.tree.count"
outputPrefix="/scratch/cqs/shengq2/spcount/RA_97_93"

In [48]:
groups_df=pd.read_csv(groupFile, sep="\t", header=None)
groups_df.rename(columns={groups_df.columns[0]:"sample", groups_df.columns.values[1]:"group"}, inplace=True)
groups_df.head()


,0,1,2
0,GF1,GermFree,all
1,GF2,GermFree,all
2,GF3,GermFree,all
3,GF4,GermFree,all
4,GF5,GermFree,all


In [36]:
tree_data = pd.read_csv(treeFile, sep="\t")
samples = tree_data.columns[3:]
print(samples)

tree_data.dtypes

Index(['GF1', 'GF2', 'GF3', 'GF4', 'GF5', 'GF6', 'GF7', 'GF8', 'GF9',
       'GermFree_Rep1', 'GermFree_Rep2', 'GermFree_Rep3', 'GermFree_Rep4',
       'GermFree_Rep5', 'GermFree_Rep6', 'GermFree_Rep7', 'SPF10', 'SPF11',
       'SPF12', 'SPF_Rep1', 'SPF_Rep3', 'SPF_Rep4'],
      dtype='object')


Rank_name        object
TaxonomyId        int64
Rank             object
GF1               int64
GF2               int64
GF3               int64
GF4               int64
GF5               int64
GF6               int64
GF7               int64
GF8               int64
GF9               int64
GermFree_Rep1     int64
GermFree_Rep2     int64
GermFree_Rep3     int64
GermFree_Rep4     int64
GermFree_Rep5     int64
GermFree_Rep6     int64
GermFree_Rep7     int64
SPF10             int64
SPF11             int64
SPF12             int64
SPF_Rep1          int64
SPF_Rep3          int64
SPF_Rep4          int64
dtype: object

In [7]:
import subprocess

for ind in range(3, tree_data.shape[1]):
  sample = tree_data.columns[ind]
  print(sample)
  subprocess.call(['ktImportTaxonomy', '-o', f"{outputPrefix}.{sample}.html", '-s', f"{ind+1}", treeFile, sample])
  break


GF1
Loading taxonomy...
Importing /scratch/cqs/shengq2/spcount/RA_97_93.tree.count...
Importing GF1...


[ ERROR ]  Couldn't open "GF1".


In [10]:
groups = groups_df.group.unique()
groups

array(['GermFree', 'SPF'], dtype=object)

In [44]:
group_data=tree_data.iloc[:, [0,1,2]]
group_data.head()

,Rank_name,TaxonomyId,Rank
0,Pseudomonas,286,genus
1,Bacteria,2,superkingdom
2,Gammaproteobacteria,1236,class
3,Proteobacteria,1224,phylum
4,Burkholderiales,80840,order


In [45]:
for gf in groups_df.groupby('group'):
  gname = gf[0]
  gdf = gf[1]
  gsamples = gdf['sample'].tolist()
  print(gname)
  print(gsamples)
  
  group_data[gname] = tree_data[gsamples].sum(axis=1)

group_data.head()

GermFree
['GF1', 'GF2', 'GF3', 'GF4', 'GF5', 'GF6', 'GF7', 'GF8', 'GF9', 'GermFree_Rep1', 'GermFree_Rep2', 'GermFree_Rep3', 'GermFree_Rep4', 'GermFree_Rep5', 'GermFree_Rep6', 'GermFree_Rep7']
SPF
['SPF10', 'SPF11', 'SPF12', 'SPF_Rep1', 'SPF_Rep3', 'SPF_Rep4']


/tmp/ipykernel_18276/4047147982.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_data[gname] = tree_data[gsamples].sum(axis=1)


,Rank_name,TaxonomyId,Rank,GermFree,SPF
0,Pseudomonas,286,genus,25199253,8230656
1,Bacteria,2,superkingdom,16734990,6279763
2,Gammaproteobacteria,1236,class,6067947,1610803
3,Proteobacteria,1224,phylum,4900529,1576646
4,Burkholderiales,80840,order,1322304,381373


In [46]:
group_data.to_csv(outputPrefix + ".group.txt", sep="\t", index=None)